## Solveur SAT

In [1006]:
MAX_GRADE = 21

In [1007]:
size = 100
nb_grades = 5
noise = 0
nb_class = 1

s = {i for i in range(nb_grades)}

s = frozenset(s)

gen = GradesGenerator(size=size, nb_grades=nb_grades,noise=noise, nb_class=nb_class)

grades, admissions = gen.generate_grades()

gen.analyze_gen()


if nb_class == 1:
    admissions = admissions.astype(int)


---------Analyze----------
Lambda: 0.35204820646238166
Weights: [0.03840765 0.40847839 0.0867935  0.07018705 0.39613341]
Betas: [14  9 11 11 11]
Got-in: {True: 81, False: 19}
% Got-in: {True: 81.0, False: 19.0} %
--------------------------


In [1008]:
#Variable aplha
alpha = []
for i in range(nb_grades):
    for k in range(MAX_GRADE):
        alpha.append((i,k)) # ==> donne tous les alpha i k 

#Dictionnaire alpha
v2i_alpha = {v : i+1 for i,v in enumerate(alpha)} # à chaque variable associe un nombre
A = len(v2i_alpha)


In [1009]:
from itertools import chain
from itertools import combinations

#Créer l'ensemble des subset 
def powerset(iterable): 
    s = list(iterable)
    return( chain.from_iterable(combinations(s, r) for r in range(len(s)+1)) )

#L'ensemble des subset 
subsets = list(powerset(s)) 


#Dictionnaire beta
v2i_beta = {frozenset(v) : A+i+1 for i,v in enumerate(subsets)}


In [1010]:
#Clause 1
clause_1 = []

for i in range(nb_grades):
    for k in range(0,MAX_GRADE-1):
        for j in range(k+1,MAX_GRADE):
            clause_1.append([-v2i_alpha[(i,k)], v2i_alpha[(i,j)]])
         

In [1011]:
#Clause 2
clause_2 = []

for i in subsets:
    C_prime = frozenset(i)
    for j in subsets:
        C = frozenset(j)
        if C.issubset(C_prime) and C != C_prime:
            clause_2.append([-v2i_beta[C], v2i_beta[C_prime]])



In [1012]:
#Clause 3

clause_3 = []
for st_idx,student in enumerate(grades):
    if admissions[st_idx] == 1:
        for c in subsets:
            alpha = []
            for i in c:
                alpha.append(v2i_alpha[(i,student[i])])
        
            clause_3.append(alpha+[v2i_beta[frozenset(s.difference(c))]])

clause_3

[[137],
 [20, 136],
 [27, 135],
 [52, 134],
 [75, 133],
 [96, 132],
 [20, 27, 131],
 [20, 52, 130],
 [20, 75, 129],
 [20, 96, 128],
 [27, 52, 127],
 [27, 75, 126],
 [27, 96, 125],
 [52, 75, 124],
 [52, 96, 123],
 [75, 96, 122],
 [20, 27, 52, 121],
 [20, 27, 75, 120],
 [20, 27, 96, 119],
 [20, 52, 75, 118],
 [20, 52, 96, 117],
 [20, 75, 96, 116],
 [27, 52, 75, 115],
 [27, 52, 96, 114],
 [27, 75, 96, 113],
 [52, 75, 96, 112],
 [20, 27, 52, 75, 111],
 [20, 27, 52, 96, 110],
 [20, 27, 75, 96, 109],
 [20, 52, 75, 96, 108],
 [27, 52, 75, 96, 107],
 [20, 27, 52, 75, 96, 106],
 [137],
 [13, 136],
 [38, 135],
 [45, 134],
 [71, 133],
 [89, 132],
 [13, 38, 131],
 [13, 45, 130],
 [13, 71, 129],
 [13, 89, 128],
 [38, 45, 127],
 [38, 71, 126],
 [38, 89, 125],
 [45, 71, 124],
 [45, 89, 123],
 [71, 89, 122],
 [13, 38, 45, 121],
 [13, 38, 71, 120],
 [13, 38, 89, 119],
 [13, 45, 71, 118],
 [13, 45, 89, 117],
 [13, 71, 89, 116],
 [38, 45, 71, 115],
 [38, 45, 89, 114],
 [38, 71, 89, 113],
 [45, 71, 89, 11

In [1013]:
clause_4 = []


for st_idx,student in enumerate(grades):
    if admissions[st_idx] == 0:
        for c in subsets:
            alpha = []
            for i in c:
                alpha.append(-v2i_alpha[(i,student[i])])
        
            clause_4.append(alpha+[-v2i_beta[frozenset(c)]])


In [1014]:
#Construction du DIMCS et Résolution
i2v = {}

for i in range(len(v2i_alpha)):
    i2v[i+1] = list(v2i_alpha.keys())[list(v2i_alpha.values()).index(i+1)]

for i in range(len(v2i_beta)):
    i2v[i+1+A] = list(v2i_beta.keys())[list(v2i_beta.values()).index(i+A+1)]

import subprocess

def clauses_to_dimacs(clauses,numvar) :
    dimacs = 'c This is it\np cnf '+str(numvar)+' '+str(len(clauses))+'\n'
    for clause in clauses :
        for atom in clause :
            dimacs += str(atom) +' '
        dimacs += '0\n'
    return dimacs

def write_dimacs_file(dimacs, filename):
    with open(filename, "w", newline="") as cnf:
        cnf.write(dimacs)

#Attention à utiliser la vesion du solveur compatible avec votre système d'exploitation, mettre le solveur dans le même dossier que ce notebook        

def exec_gophersat(filename, cmd = "./gophersat.exe", encoding = "utf8") :
    result = subprocess.run([cmd, filename], stdout=subprocess.PIPE, check=True, encoding=encoding)
    string = str(result.stdout)
    lines = string.splitlines()

    if lines[1] != "s SATISFIABLE":
        return False, [], {}

    model = lines[2][2:].split(" ")

    return True, [int(x) for x in model if int(x) != 0], { i2v[abs(int(v))] : int(v) > 0 for v in model if int(v)!=0} 


In [1015]:
#Lancer la résolution

myClauses= clause_1 + clause_2 + clause_3 + clause_4
myDimacs = clauses_to_dimacs(myClauses,len(i2v))

write_dimacs_file(myDimacs,"./workingfile.cnf")
res = exec_gophersat("./workingfile.cnf")

#Résultat
print(res)

(True, [-1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, -22, -23, -24, -25, -26, -27, -28, -29, -30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, -43, -44, -45, -46, -47, -48, -49, -50, -51, -52, -53, -54, -55, -56, 57, 58, 59, 60, 61, 62, 63, -64, -65, -66, -67, -68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, -85, -86, -87, -88, -89, -90, -91, -92, -93, -94, -95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, -106, -107, -108, -109, -110, 111, 112, -113, -114, 115, -116, 117, 118, -119, 120, 121, 122, 123, 124, -125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137], {(0, 0): False, (0, 1): True, (0, 2): True, (0, 3): True, (0, 4): True, (0, 5): True, (0, 6): True, (0, 7): True, (0, 8): True, (0, 9): True, (0, 10): True, (0, 11): True, (0, 12): True, (0, 13): True, (0, 14): True, (0, 15): True, (0, 16): True, (0, 17): True, (0, 18): True, (0, 19): True, (0, 20): True, (1, 0): False, (1, 1): False, (1, 2): False, (1, 3): Fals

In [1016]:
_, variables_idx, d = res

In [1017]:
predicted = []
for student in grades:
    validated_courses = set()
    for i,k in enumerate(student):
        if d[(i,k)]:
            validated_courses.add(i)
    validated_courses = frozenset(validated_courses)
    if d[validated_courses]:
        predicted.append(1)
    else:
        predicted.append(0)
predicted
        


[1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

In [ ]:
from sklearn.metrics import f1_score, accuracy_score

print(accuracy_score(admissions,predicted))
print(f1_score(admissions,predicted))

1.0
1.0


SAT Solver Multi classes

In [ ]:
import sys
sys.path.append('../')

from generator import GradesGenerator

In [ ]:
size = 100
nb_grades = 5
noise = 0
nb_class = 1

s = {i for i in range(nb_grades)}

s = frozenset(s)

gen = GradesGenerator(size=size, nb_grades=nb_grades,noise=noise, nb_class=nb_class)

grades, admissions = gen.generate_grades()


if nb_class == 1:
    admissions = admissions.astype(int)


In [ ]:
#Variable aplha
alpha = []
for i in range(nb_grades):
    for k in range(MAX_GRADE):
        for h in range(nb_class+1):
            alpha.append((i,k,h)) # ==> donne tous les alpha i k h

#Dictionnaire alpha
v2i_alpha = {v : i+1 for i,v in enumerate(alpha)} # à chaque variable associe un nombre
A = len(v2i_alpha)


In [ ]:
from itertools import chain
from itertools import combinations

#Créer l'ensemble des subset 
def powerset(iterable): 
    s = list(iterable)
    return( chain.from_iterable(combinations(s, r) for r in range(len(s)+1)) )

#L'ensemble des subset 
subsets = list(powerset(s)) 


#Dictionnaire beta
v2i_beta = {frozenset(v) : A+i+1 for i,v in enumerate(subsets)}


In [ ]:
clause_1 = []

for i in range(nb_grades):
    for k in range(MAX_GRADE-1):
        for h in range(nb_class):
            for j in range(k+1,MAX_GRADE):
                clause_1.append([-v2i_alpha[(i,k,h)], v2i_alpha[(i,j,h)]])

In [ ]:
clause_2 = []

for i in subsets:
    C_prime = frozenset(i)
    for j in subsets:
        C = frozenset(j)
        if C.issubset(C_prime) and C != C_prime:
            clause_2.append([-v2i_beta[C], v2i_beta[C_prime]])


In [ ]:
clause_3 = []


clause_3 = []
for st_idx,student in enumerate(grades):
    for c in subsets:
        alpha = []
        for i in c:
            alpha.append(v2i_alpha[(i,student[i],admissions[st_idx])])
    
        clause_3.append(alpha+[v2i_beta[frozenset(s.difference(c))]])





In [ ]:
clause_4 = []

for st_idx,student in enumerate(grades):
    if admissions[st_idx] < nb_class:
        for c in subsets:
            alpha = []
            for i in c:
                alpha.append(-v2i_alpha[(i,student[i],admissions[st_idx]+1)])
        
            clause_4.append(alpha+[-v2i_beta[frozenset(c)]])


In [ ]:
clause_5 = []

for k in range(MAX_GRADE):
    for i in range(nb_grades):
        for h in range(nb_class):
            for j in range(h+1, nb_class+1):
                clause_5.append([v2i_alpha[(i,k,h)],-v2i_alpha[(i,k,j)]])



In [ ]:
#Construction du DIMCS et Résolution
i2v = {}

for i in range(len(v2i_alpha)):
    i2v[i+1] = list(v2i_alpha.keys())[list(v2i_alpha.values()).index(i+1)]

for i in range(len(v2i_beta)):
    i2v[i+A+1] = list(v2i_beta.keys())[list(v2i_beta.values()).index(i+1+A)]

import subprocess

def clauses_to_dimacs(clauses,numvar) :
    dimacs = 'c This is it\np cnf '+str(numvar)+' '+str(len(clauses))+'\n'
    for clause in clauses :
        for atom in clause :
            dimacs += str(atom) +' '
        dimacs += '0\n'
    return dimacs

def write_dimacs_file(dimacs, filename):
    with open(filename, "w", newline="") as cnf:
        cnf.write(dimacs)

#Attention à utiliser la vesion du solveur compatible avec votre système d'exploitation, mettre le solveur dans le même dossier que ce notebook        

def exec_gophersat(filename, cmd = "./gophersat.exe", encoding = "utf8") :
    result = subprocess.run([cmd, filename], stdout=subprocess.PIPE, check=True, encoding=encoding)
    string = str(result.stdout)
    lines = string.splitlines()

    if lines[1] != "s SATISFIABLE":
        return False, [], {}

    model = lines[2][2:].split(" ")

    return True, [int(x) for x in model if int(x) != 0], { i2v[abs(int(v))] : int(v) > 0 for v in model if int(v)!=0} 


In [ ]:
myClauses= clause_1 + clause_2 + clause_3 + clause_4 + clause_5
myDimacs = clauses_to_dimacs(myClauses,len(i2v))

write_dimacs_file(myDimacs,"./workingfile_multi.cnf")
res = exec_gophersat("./workingfile_multi.cnf")

#Résultat

In [ ]:
_, variables_idx, d = res

In [ ]:
predicted = []
for student in grades:
    current_class = 0
    for h in range(nb_class+1):
        current_class = h
        validated_courses = set()
        for i,k in enumerate(student):
            if d[(i,k,h)]:
                validated_courses.add(i)
        validated_courses = frozenset(validated_courses)
        if validated_courses in d.keys():
            if d[validated_courses]:
                continue
            else:
                break
        else:
            break
    predicted.append(current_class-1)

predicted
        


[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [ ]:
from sklearn.metrics import f1_score, accuracy_score

print(accuracy_score(admissions,predicted))
print(f1_score(admissions,predicted, average='macro'))

0.54
0.35064935064935066
